# 7.18 Gas

A natural gas pipeline (schedule 20 14"") is 100 miles long. Inlet pressure is 1300 psia, and outlet pressure is 300 psia.

The average temperature is 40 deg F. The gas composition is 75% methane, 21% ethane, and 4 % propane.

Find the flow rate in MMscfd.


In [1]:
from fluids.units import *
from math import pi
P1 = 1300*u.psi
P2 = 300*u.psi
T = 40*u.degF
L = 100*u.miles

mu = 1.1e-5*u.Pa # 
# mu = 1.915E-5*u.Pa # A more correct value, but hinders matching the problem
NPS, Di, Do, t = nearest_pipe(NPS=14, schedule='20')
A = 0.25*pi*Di**2

In [2]:
from thermo import *
from thermo import PRMIX
Tcs = [190.564, 305.32, 369.83]
Pcs = [4599000.0, 4872000.0, 4248000.0]
omegas = [0.008, 0.098, 0.152]
MWs = [16.04246, 30.06904, 44.09562]
zs = [0.75, 0.21, .04]
MW = sum(zs[i]*MWs[i] for i in range(3))*u.g/u.mol

In [3]:
eos_1 = PRMIX(T=T.to(u.K).magnitude, P=P1.to(u.Pa).magnitude, zs=zs, Tcs=Tcs, Pcs=Pcs, omegas=omegas)
eos_2 = PRMIX(T=T.to(u.K).magnitude, P=P2.to(u.Pa).magnitude, zs=zs, Tcs=Tcs, Pcs=Pcs, omegas=omegas)
eos_std = PRMIX(T=288.15, P=101325.0, zs=zs, Tcs=Tcs, Pcs=Pcs, omegas=omegas)

Vm1 = eos_1.V_g*u.m**3/u.mol
rho1 = (Vm1)**-1*MW

Vm2 = eos_2.V_g*u.m**3/u.mol
rho2 = (Vm2)**-1*MW

Vm_std = eos_std.V_g*u.m**3/u.mol
rho_std = (Vm_std)**-1*MW

roughness = 0.0018*u.inch

rho = 0.5*(rho1 + rho2)

v = 10.0 # Initial guess for velocity
Re = rho*v*Di/mu
fd = friction_factor(Re=Re, eD=roughness/Di)

for i in range(8):
    # Solve for velocity with sequential substitution
    m = isothermal_gas(rho, fd, P1=P1, P2=P2, L=L, D=Di)
    v = m/(A*rho)
    Re = rho*v*Di/mu
    fd = friction_factor(Re=Re, eD=roughness/Di)
Q = v*A
correction = rho_std/rho

# pint does not support mmscfd
mmscfd = Q.to(u.ft**3/u.day).magnitude/1e6/correction
print('According to the full isothermal equation, the flowrate is %g MMscfd' %(mmscfd))
# Q.to(u.m**3/u.s)

According to the full isothermal equation, the flowrate is 103.434 MMscfd


In [4]:
# Note Z_avg should be used in the Panhandle and Weymouth equations
# However Crane omits them; they are not used here to match Crane.
Z_avg = 0.5*(eos_1.Z_g + eos_2.Z_g)
MW_air = 28.966*u.g/u.mol
SG = MW/MW_air

In [5]:
# Crane does not use the efficiency term on Weymouth
Q_Weymouth = Weymouth(SG, Tavg=T, L=L, D=Di, P1=P1, P2=P2, Zavg=1, E=1)

mmscfd = Q_Weymouth.to(u.ft**3/u.day).magnitude/1e6
print('According to the Weymouth equation, the flowrate is %g MMscfd' %(mmscfd))

According to the Weymouth equation, the flowrate is 105.047 MMscfd


In [6]:
Q_panhandle = Panhandle_A(SG, Tavg=T, L=L, D=Di, P1=P1, P2=P2, Zavg=1, E=0.92)

mmscfd = Q_panhandle.to(u.ft**3/u.day).magnitude/1e6
print('According to the Panhandle A equation, the flowrate is %g MMscfd' %(mmscfd))

According to the Panhandle A equation, the flowrate is 128.143 MMscfd


Besides the simplifications Crane makes, the methods are fair approximations. 
The values in Crane are 107.8, 105.1, and 128.2 mmscfd respectively. The isothermal calculation employed by Crane is a simplified one, explaining the difference - it does not account for expansion rigorously.